#**Tennis Shots Identification and Counting using YOLOv7 Pose Estimation**

#**Clone repo and install Flask**



In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn python-multipart

In [ ]:
!git clone https://github.com/galleon/yellow_bumblebee

#**Downloading the YOLO v7 and trained LSTM model file**

In [ ]:
%cd /content/yellow_bumblebee

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt -P .

In [ ]:
!wget https://github.com/XaLeroy/tennis_vision/raw/main/VideoInput/video_input2.mp4 -P .

In [ ]:
!wget https://github.com/sahibjotsaggu/San-Francisco-Pro-Fonts/raw/master/SF-Pro.ttf -O sfpro.ttf

### Upload video file

In [ ]:
from google.colab import files

vf = files.upload()

In [ ]:
vf_name = list(vf.keys())[0]

### Upload mmodelfinal.h5

In [ ]:
_ = files.upload()

## Run code

In [ ]:
from tennis_shot_identification_and_counts import run
run(source=vf_name, device="0")

## Show result

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = f'/content/yellow_bumblebee/{vf_name}'

# Compressed video path
compressed_path = "/content/yellow_bumblebee/result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Create FastAPI endpoint using ngrok

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse, StreamingResponse
import cv2
from PIL import Image
import io
import tempfile
import os
from pathlib import Path
from fastapi.middleware.cors import CORSMiddleware

from tennis_shot_identification_and_counts import run

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.post("/process_video")
async def process_video(file: UploadFile = File(...)):

    _, file_extension = os.path.splitext(file.filename)

    with tempfile.NamedTemporaryFile(delete=True, suffix=file_extension) as temp_file:
        temp_file.write(await file.read())
        temp_file.seek(0)

        # for x in os.listdir():
        #   print(x)

        print(temp_file.name)

        run(source=temp_file.name, device="0")

        # Test if the video file exists and can be read
        try:
            output_video_path = Path(temp_file.name.split(".")[0] + "_wed.mp4")
            if output_video_path.exists() and output_video_path.is_file():
                return FileResponse(str(output_video_path), media_type="video/mp4", filename=output_video_path.name)
        finally:
            if output_video_path.exists():
                output_video_path.unlink()

    # If the process fails, return an error response
    return {"error": "Failed to process the video"}


In [ ]:
from google.colab import userdata
ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok.set_auth_token(ngrok_authtoken)

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)